In [3]:
import numpy as np
import pandas as pd
import numpy
import matplotlib.pyplot as plt

#Fix colormath depreciated numpy method
def patch_asscalar(a):
    return a.item()
setattr(numpy, "asscalar", patch_asscalar)

#Import data
all_colors = pd.read_csv("data/all-colors.csv")
chosen_colors = pd.read_csv("data/chosen-colors.csv")

#Remove chosen colors
condition = all_colors["Shirt Color Name"].isin(chosen_colors["Shirt Color"])
remaining_colors = all_colors.drop(all_colors[condition].index, inplace=False)

#Confirm removal
# print(len(remaining_colors))
# print(len(all_colors))
# print(len(chosen_colors))

# Find Most Distant Colors to Chosen Colors

In [4]:
import colormath.color_objects as co
import colormath.color_diff as cdiff

def compute_average_distance(row):
    color_Lab = None

    color_Lab = co.LabColor(*map(float, row["L*a*b Value"].split(", ")))

    distances = []

    #Compute all distances from chosen colors
    for index, row_remaining in chosen_colors.iterrows():
        other_color = co.LabColor(*map(float, row_remaining["L*a*b Value"].split(", ")))

        #Newer, asymmetric formula, weighted for fabrics, using
        distance = cdiff.delta_e_cmc(other_color, color_Lab , pl=2, pc=1)

        #Older symmetric formula, weighted for fabrics
        #distance = cdiff.delta_e_cie2000(color_Lab, other_color,Kl=2, Kc=1, Kh=1)

        distances += [distance]

    average_distance = sum(distances)/len(remaining_colors)

    row["Distance"] = average_distance
    return row

remaining_colors = remaining_colors.apply(compute_average_distance, axis=1)

In [5]:
sorted_remaining = remaining_colors.sort_values(by = "Distance", ascending=False)
sorted_remaining.iloc[:].head(10)

,Shirt Color Name,PMS Color Code,L*a*b Value,Hex,Distance
37,Poppy,2347C,"49.45, 75.12, 67.21",E10600,13.722151
24,Maize Yellow,7548CP,"83.14, 5.01, 85.02",F2CA00,12.989638
34,Orange,1645C,"65.69, 58.38, 58.65",FF6A39,11.935688
83,Heather Orange,1645 C,"65.69, 58.38, 58.65",FF6A39,11.935688
73,Heather Kelly,340 C,"51.80, -65.36, 14.55",00965E,11.553796
47,Teal,3262C,"66.44, -59.33, -12.05",00BFB2,11.106415
104,Heather Yellow,100 CP,"92.09, -6.83, 62.42",F6EB69,10.961603
9,Burnt Orange,159C,"54.43, 43.54, 58.62",CB6015,10.749025
95,Heather Red,193C,"42.79, 66.14, 27.76",BF0D3E,10.282757
105,Heather Yellow Gold,134 C,"86.67, 8.31, 54.63",FDD26E,9.848363


In [6]:
chosen_colors.to_json(orient="records")

'[{"Shirt Color":"Heather Ice Blue","Pantone Color":"317 CP","L*a*b Value":"85.65, -11.38, -2.20","HEX":"BDDDD9"},{"Shirt Color":"Berry","Pantone Color":"233 CP","L*a*b Value":"45.38, 71.84, -7.30","HEX":"C6057B"},{"Shirt Color":"Oxblood Black","Pantone Color":"7449 CP","L*a*b Value":"28.13, 16.23, -6.49","HEX":"55394D"},{"Shirt Color":"White","Pantone Color":"White C","L*a*b Value":"100, 0, 0","HEX":"FFFFFF"},{"Shirt Color":"Ash","Pantone Color":"428 CP","L*a*b Value":"78.50, -2.14, -0.31","HEX":"BDC4C2"},{"Shirt Color":"Gold","Pantone Color":"137 C","L*a*b Value":"76.47, 31.14, 81.64","HEX":"FFA400"},{"Shirt Color":"Yellow","Pantone Color":"100 CP","L*a*b Value":"92.09, -6.83, 62.42","HEX":"F6EB69"},{"Shirt Color":"Deep Teal","Pantone Color":"3025 CP","L*a*b Value":"32.45, -23.19, -27.24","HEX":"005876"},{"Shirt Color":"Heather Maroon","Pantone Color":"7644 CP","L*a*b Value":"27.76, 25.21, -2.86","HEX":"603147"},{"Shirt Color":"Evergreen","Pantone Color":"343 C","L*a*b Value":"31.90,

In [7]:
#Orange to Gold

orange = all_colors[all_colors["Shirt Color Name"] == "Orange"]
gold = all_colors[all_colors["Shirt Color Name"] == "Gold"]
burnt_orange = all_colors[all_colors["Shirt Color Name"] == "Burnt Orange"]

comparison_list = [gold, orange, burnt_orange]

comparison_matrix = [[0]*3 for i in range(3)]

for i in range(len(comparison_list)):
    for j in range(len(comparison_matrix)):
        #I really should just fix the tables
        target_color = co.LabColor(*map(float,comparison_list[i].iloc[0]["L*a*b Value"].split(", ")))
        comparison_color = co.LabColor(*map(float,comparison_list[j].iloc[0]["L*a*b Value"].split(", ")))

        #Newer formula
        distance = cdiff.delta_e_cmc(target_color, comparison_color, pl=2, pc=1)

        comparison_matrix[i][j] = distance

print("    Gold    Orange    Burnt Orange")
for row in comparison_matrix:
    print(row)

    Gold    Orange    Burnt Orange
[0.0, 24.24013342020266, 17.480586695643115]
[26.402279861193914, 0.0, 9.887678740145626]
[22.286845806880518, 11.611165190695397, 0.0]


In [8]:
import networkx as nx

# Let's make an adjacency matrix representing color distances to/from each other
adjacency_matrix = pd.DataFrame(np.zeros(shape=(16, 16)),
columns = chosen_colors["Shirt Color"].unique(), index=chosen_colors["Shirt Color"].unique())

for index_a, row_a in chosen_colors.iterrows():
    for index_b, row_b in chosen_colors.iterrows():
        color_a = co.LabColor(*map(float, row_a["L*a*b Value"].split(", ")))
        color_b = co.LabColor(*map(float, row_b["L*a*b Value"].split(", ")))

        distance = cdiff.delta_e_cie2000(color_a, color_b, Kl=2, Kc=1, Kh=1)

        adjacency_matrix.at[row_a["Shirt Color"], row_b["Shirt Color"]] = distance


# Plot the Chosen Colors

In [9]:
import plotly.express as px

import plotly.express as px
import plotly.graph_objects as go

# Assuming your CSV has columns: "Shirt Color", "Pantone Color", "L*a*b Value", "HEX"
df = pd.read_csv('data/chosen-colors.csv')

sorted_fixed_columns = sorted_remaining.drop('Distance', axis=1)

sorted_fixed_columns.columns = df.columns

def prepend_star(row):
    row["Shirt Color"] = "*" + row["Shirt Color"]
    return row

sorted_fixed_columns = sorted_fixed_columns.apply(prepend_star, axis=1)

sorted_fixed_columns.head()

df.head()
df = pd.concat([df, sorted_fixed_columns.iloc[0:10]], ignore_index=True)


# Parse the L*a*b values from the string format
df[['L', 'a', 'b']] = df['L*a*b Value'].str.split(', ', expand=True).astype(float)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=df['a'],        # a* axis (green-red)
    y=df['L'],        # L* axis (lightness)
    z=df['b'],        # b* axis (blue-yellow)
    mode='markers+text',
    text=df['Shirt Color'],
    hovertemplate=
        '<b>%{text}</b><br>' +
        'L*: %{y:.1f}<br>' +
        'a*: %{x:.1f}<br>' +
        'b*: %{z:.1f}<br>',
    marker=dict(
        size=10,
        color=['#' + hex.strip() for hex in df['HEX']],  # Use actual hex colors
        opacity=0.8
    ),
    textposition='top center'
)])

# Update the layout
fig.update_layout(
    title='Colors in LAB Space',
    scene=dict(
        xaxis_title='a* (green to red)',
        yaxis_title='L* (black to white)',
        zaxis_title='b* (blue to yellow)',
        # Set appropriate ranges for each axis
        xaxis=dict(range=[-128, 128]),
        yaxis=dict(range=[0, 100]),
        zaxis=dict(range=[-128, 128])
    ),
    width=1000,
    height=800,
    showlegend=False
)

# Add better camera angle
fig.update_layout(scene_camera=dict(
    up=dict(x=0, y=1, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.5, y=1.5, z=1.5)
))

# Show the plot
#fig.show()

fig.write_html("lab_color_space_ECMC.html")

In [10]:
df.head(20)

,Shirt Color,Pantone Color,L*a*b Value,HEX,L,a,b
0,Heather Ice Blue,317 CP,"85.65, -11.38, -2.20",BDDDD9,85.65,-11.38,-2.20
1,Berry,233 CP,"45.38, 71.84, -7.30",C6057B,45.38,71.84,-7.30
2,Oxblood Black,7449 CP,"28.13, 16.23, -6.49",55394D,28.13,16.23,-6.49
3,White,White C,"100, 0, 0",FFFFFF,100.00,0.00,0.00
4,Ash,428 CP,"78.50, -2.14, -0.31",BDC4C2,78.50,-2.14,-0.31
5,Gold,137 C,"76.47, 31.14, 81.64",FFA400,76.47,31.14,81.64
6,Yellow,100 CP,"92.09, -6.83, 62.42",F6EB69,92.09,-6.83,62.42
7,Deep Teal,3025 CP,"32.45, -23.19, -27.24",005876,32.45,-23.19,-27.24
8,Heather Maroon,7644 CP,"27.76, 25.21, -2.86",603147,27.76,25.21,-2.86
9,Evergreen,343 C,"31.90, -26.89, 6.33",115740,31.90,-26.89,6.33


In [11]:
from ian_lab_graph import test
from ian_lab_graph import ian_generate_lab_3d
# df = pd.read_csv('data/chosen-colors.csv')
#
# ian_generate_lab_3d(df=df, figure_title="Chosen Colors", output_file_name="chosen_colors_3d")
test()

NameError: name 'go' is not defined